In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
 
import torchvision  # 图片、视频处理
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os, shutil

device = 'cuda' if torch.cuda.is_available else 'cpu'
# device='cpu'
torch.__version__, torchvision.__version__, device


In [ ]:
torch.cuda.caching_allocator_delete
torch.cuda.empty_cache()  # 释放显存
print('Memory Allocated', torch.cuda.memory_allocated() )

In [ ]:
torchvision.datasets.ImageFolder
# train_ds = torchvision.datasets.MNIST('data', 
#                                       train=True, 
#                                       download=True, 
#                                       transform=ToTensor())

In [ ]:
# 建立项目主目录
base_dir = r'./datasets/4weather'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

if not os.path.isdir(base_dir): os.mkdir( base_dir)
if not os.path.isdir(train_dir): os.mkdir( train_dir )
if not os.path.isdir(test_dir): os.mkdir( test_dir )

# 建立子目录
specises = ['cloudy', 'rain', 'shine', 'sunrise']
for train_or_test in ['train', 'test']:
    for spec in specises:
        sub_dir = base_dir + '/' + train_or_test + '/' + spec
        if not os.path.isdir(sub_dir): 
            os.mkdir( sub_dir)
            print(sub_dir, ' is created.')
        else:
            print(sub_dir, ' is existed.')
    

In [ ]:
""" 整理文件的工作目录"""
image_dir=r'./datasets/weather_imgs/'
# os.listdir( image_dir)

for i, img in enumerate( os.listdir(image_dir) ):
    for spec in specises:
        if spec in img:
            s = os.path.join(image_dir, img)
            if i%5 == 0:
                d = os.path.join(base_dir, 'test', spec, img)
            else:
                d = os.path.join(base_dir, 'train', spec, img)
            shutil.copy(s, d)

# 文件存放状况
for train_or_test in ['train', 'test']:
    for spec in specises:
        # print(  os.listdir( os.path.join(base_dir, train_or_test, spec)), '\n')
        print( train_or_test, spec, len(os.listdir( os.path.join(base_dir, train_or_test, spec))) )

In [ ]:
transform = transforms.Compose([
    transforms.Resize((96, 96)),
    transforms.ToTensor(),                           # 1、Channel放前面，2、转换成Tensor，3、数据归一化到0-1之间。
    transforms.Normalize( mean=[0.5, 0.5, 0.5], 
                         std=[0.5, 0.5, 0.5],
                         )
])

In [ ]:
train_ds = torchvision.datasets.ImageFolder(
    train_dir,
    transform=transform
)
test_ds = torchvision.datasets.ImageFolder(
    test_dir,
    transform=transform
)

# test_ds = test_ds.to(device)

In [ ]:
train_ds

In [ ]:
train_ds.classes

In [ ]:
train_ds.class_to_idx

In [ ]:
len(train_ds), len(test_ds)

In [ ]:
""" 创建 dataloader """
BATCHSIZE = 64
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=BATCHSIZE, shuffle=True)  # batch_size开始为64,可能是最佳，或128
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=BATCHSIZE, shuffle=False)

In [ ]:
imgs, labels = next(iter(train_dl))   # train dataloder 可以分解出X和Ydata


In [ ]:
imgs.shape, labels.shape   # 1: 黑白图片， 28x28的图片分辨率， 64张


In [ ]:
# 切片：取第一条，channel 0的数据(共有3个channel，红绿蓝三色); 输出2维：96x96的图片
imgs[0].shape 


In [ ]:
im = imgs[0].permute(1,2,0)   # 更换tensor中，向量的顺序，原来是0,1,2
im.shape                       # 注意 shape 和上面的变化

In [ ]:
im_np = im.numpy() 
# im_np

In [ ]:
plt.imshow(im_np)

In [ ]:
""" 测试卷积函数 """
# input = torch.randint(2,[2,3,4])
# input = torch.randn(5)
# input
# input, torch.relu(input)

# input,torch.sigmoid( input )
# input,torch.tanh( input )
# input,nn.LeakyReLU( input )

In [ ]:
im_np.min(), im_np.max()

In [ ]:
im_np = (im+1)/2    # 从 -1 到 1 变为 0-1 之间

In [ ]:
im_np.min(), im_np.max()

In [ ]:
train_ds.class_to_idx

In [ ]:
id_to_class = dict(
    (v,k) for k, v in train_ds.class_to_idx.items()
)
id_to_class

In [ ]:
""" imgs, labels 显示 """
plt.figure(figsize=(12, 8))     # 设置画布
for i, (img, label) in enumerate( zip(imgs[:6], labels[:6]) ): #取前10张图片   
    img = img.permute(1,2,0) 
    img_np = (img.numpy() + 1)/2 
    plt.subplot(2, 3, i+1)
    plt.title( id_to_class.get( label.item() ))
    plt.imshow( img_np )


上面是数据准备和模式测试， 
以下正式开始设计：

In [ ]:
class Net( nn.Module ):
    def __init__(self):
        super(Net, self).__init__()  # 父类的初始化
        self.conv1 = nn.Conv2d(3, 16, 3)        # 3:图片channel为3，作为输入； 16个卷积核，自定义； curnel size为3x3；
        self.bn1 = nn.BatchNorm2d(16)           # 16: 上层的feature 16
        self.pool = nn.MaxPool2d(2,2)           # 设计了MaxPool方法，2x2结构；
        self.conv2 = nn.Conv2d(16, 32, 3)       # 输入=conv1的输出16， 卷积核翻倍到32， 
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, 3)       # 输入=conv1的输出32， 卷积核翻倍到64， !#! 卷积核翻倍，模仿了vgg的著名卷积结构，比较有效。
        self.bn3 = nn.BatchNorm2d(64)
        self.drop = nn.Dropout(0.5)             # 0.5, 丢掉50% 神经单元
        self.drop2d = nn.Dropout2d(0.5)            # 0.5, 丢掉50% 神经单元
        self.fc1 = nn.Linear(64*10*10, 1024)    # fc1: 全连接1层; 64*12*12为预估 (通过print输出调整)； 1024为自定义；
        self.bn_f1 = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 256)           # fc2: 全连接2层; 1024为1层输出；4为4分类输出，配合label
        self.bn_f2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 4)           # fc2: 全连接2层; 1024为1层输出；4为4分类输出，配合label
        
    def forward(self, x):
        x = self.pool( F.relu(self.conv1(x)) )
        x = self.bn1(x)                         # 在 pool 之后再添加
        x = self.pool( F.relu(self.conv2(x)) )
        x = self.bn2(x)
        x = self.pool( F.relu(self.conv3(x)) )
        x = self.bn3(x)
        
        x = self.drop2d(x)
        # print(x.size())             # torch.Size([16, 64, 10, 10])
        x = x.view(-1, 64*10*10)    # 通过view进行展平，其中64源于conv3的输出厚度64， 12x12为预估; Pytorch每次都需要我们去计划12x12，是有些问题的，不够成熟
        x = F.relu( self.fc1(x) )
        x = self.bn_f1(x)
        x = self.drop(x)
        x = F.relu( self.fc2(x) )
        x = self.bn_f2(x)
        x = self.drop(x)
        x = self.fc3(x)
        return x

model = Net()

In [ ]:
preds = model( imgs )
imgs.shape, preds.shape

In [ ]:
# 计算出preds的分类结果
preds_out = torch.argmax( preds, 1)     # 对preds的1维进行找最大数， 0维为个数


In [ ]:
# model刚init，成功率并不高，所以需要后续训练
labels, preds_out, labels - preds_out

In [ ]:
model = model.to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()

In [ ]:
"""
优化： 根据计算得到的损失，调整模型参数， 降低损失的过程；
- Adam 优化器
- SGD：优化model的参数、以及lr
"""
# opt = torch.optim.SGD(model.parameters(), lr=0.001)
optim = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
""" 
    训练 N 个 epoch， 记录每个epoch的train和test的损失、准确率。 
"""
# batch_size 非常关键：4096不准确； 64很准确。16也比较差。 ！！！
def fit(epoch, model, train_dl, test_dl ):
    correct, total, running_loss = 0, 0, 0
    model.train()   #设置成训练模式
    for x, y in train_dl:
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optim.zero_grad()
        loss.backward()
        optim.step()
        with torch.no_grad():
            y_pred = torch.argmax(y_pred, dim=1)
            correct += (y_pred==y).sum().item()
            total += y.size(0)
            running_loss += loss.item()
    epoch_loss = running_loss / len(train_dl.dataset) 
    epoch_acc = correct / total
                           
    test_correct, test_total, test_running_loss = 0, 0, 0
    model.eval()  # 设置成eval 评估测试模式， 推理模式
    with torch.no_grad():
        for x, y in test_dl:
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            # optim.zero_grad()
            # loss.backward()
            # optim.step()
            y_pred = torch.argmax(y_pred, dim=1)
            test_correct += (y_pred==y).sum().item()
            test_total += y.size(0)
            test_running_loss += loss.item()
    
    test_epoch_loss = test_running_loss / len(test_dl.dataset) 
    test_epoch_acc = test_correct / test_total
                           
    print('epoch: ', epoch,
        'loss: ', round(epoch_loss,3),
        'acc:', round(epoch_acc,3),
        'test_loss: ', round(test_epoch_loss,3),
        'test_acc:', round(test_epoch_acc,3)
        )         
    return epoch_acc, epoch_loss, test_epoch_acc,test_epoch_loss
    


In [ ]:
epochs = 30
train_loss, train_acc = [], []
test_loss,  test_acc  = [], []
for epoch in range( epochs ):
    epoch_acc, epoch_loss, epoch_test_acc, epoch_test_loss = fit( epoch, model, train_dl, test_dl)
    train_acc.append(epoch_acc)
    train_loss.append(epoch_loss)
    test_acc.append(epoch_test_acc)
    test_loss.append(epoch_test_loss)  # 记录、图表化后，观察是否会过拟合等问题
    

In [ ]:
plt.plot( range(epochs), train_loss, label='train_loss')
plt.plot( range(epochs), test_loss, label='test_loss')
plt.legend()
# train_acc

In [ ]:
plt.plot( range(epochs), train_acc, label='train_acc')
plt.plot( range(epochs), test_acc, label='test_acc')
plt.legend()